
# Demystifying Approximate Bayesian Computation

#### Brett Morris

### In this tutorial

We will write our own rejection sampling algorithm to approximate the posterior distributions for some fitting parameters.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import anderson_ksamp
from corner import corner

First, let's generate a series of observations $y_\mathrm{obs}$, taken at times $x$. The observations will be drawn from one of two Gaussian distributions with a fixed standard deviation, separated by $3\sigma$ from one another. There will be a fraction $f$ of the total samples in the second mode of the distirbution.

In [ ]:
# Set a random seed for reproducibility
np.random.seed(42)

# Standard deviation of both normal distributions
true_std = 1

# Mean of the first normal distribution
true_mean1 = np.pi

# Mean of the second normal distribution
true_mean2 = 3 * np.pi

# Fraction of samples in second mode: this 
# algorithm works best when the fraction 
# is between [0.2, 0.8]
true_fraction = 0.3

# Third number below is the number of samples to draw:
x = np.linspace(0, 1, 500)

y_obs = np.concatenate([true_mean1 + true_std * np.random.randn(int((1-true_fraction) * len(x))), 
                        true_mean2 + true_std * np.random.randn(int(true_fraction * len(x)))])

plt.hist(y_obs, bins=50, density=True)
plt.xlabel('$y_\mathrm{obs}$', fontsize=20)
plt.show()

So how does one fit for the means and standard deviations of the bimodal distribution? Since this example is a mixture of normal distirbutions, one way is to use [Gaussian mixture models](https://dfm.io/posts/mixture-models/), but we're going to take a different approach, which we'll see is more general later. 

## Approximate Bayesian Computation

For this particular dataset, it's easy to construct a model $\mathcal{M}$ which reproduces the observations $y_\mathcal{obs}$ – the model is simply the concatenation of two normal distributions $\mathcal{M} \sim \left[\mathcal{N} \left(\mu_1, \sigma, \textrm{size=(1-f)N}\right), \mathcal{N}\left(\mu_2, \sigma, \textrm{size=}fN\right)\right]$, where the `size` argument determines the number of samples to draw from the distribution, $N$ is the total number of draws, and $f$ is the fraction of draws in the second mode. One way to *approximate* the posterior distributions of $\theta = \{\mu_1, \mu_2, \sigma, f\}$ would be to propose new parameters $\theta^*$, and only keep a running list of the parameter combinations which produce a simulated dataset $y_\mathrm{sim}$ which very closely reproduces the observations $y_\mathrm{obs}$. 

### Summary statistic: the Anderson-Darling statistic

In practice, this requires a *summary statistic*, which measures the "distance" between the simulated dataset $y_\mathrm{sim}$ and the observations $y_\mathrm{obs}$. In this example we need a metric which measures the probability that two randomly-drawn samples $y$ are drawn from the same distribution. One such metric is the [Anderson-Darling statistic](https://en.wikipedia.org/wiki/Anderson–Darling_test), which approaches a minimum near $A^2=-1.3$ for two sets $y$ that are drawn from indistinguishable distributions, and grows to $A^2 > 10^5$ for easily distinguishable distributions.

We can see how the Anderson-Darling statistic behaves in this simple example below: 

In [ ]:
n_samples = 10000

# Generate a bimodal distribution
a = np.concatenate([np.random.randn(n_samples), 
                    3 + np.random.randn(n_samples//2)])

# Plot the bimodal distribution
fig, ax = plt.subplots(1, 2, figsize=(7, 3))
ax[0].hist(a, color='silver', range=[-4, 11], bins=50,
           lw=2, histtype='stepfilled')

# For a set of bimodal distributions with varing means: 
for mean in [0, 1.2, 5]: 
    # Generate a new bimodal distribution
    c = mean + np.concatenate([np.random.randn(n_samples), 
                               3 + np.random.randn(n_samples//2)])
    
    # Measure, plot the Anderson-darling statistic
    a2 = anderson_ksamp([a, c]).statistic
    ax[0].hist(c, histtype='step', range=[-4, 11], 
               bins=50, lw=2)

    ax[1].plot(mean, a2, 'o')

ax[0].set(xlabel='Samples', ylabel='Frequency')
ax[1].set(xlabel='Mean', ylabel='$A^2$')

fig.tight_layout()

In the figure above, we have a set of observations $y_\mathrm{obs}$ (left, gray) which we're comparing to the set of simulated observations $y_\mathrm{sim}$ (left, colors). The Anderson-Darling statistic $A^2$ is plotted for each pair of the observations and the simulations (right). You can see that the minimum of $A^2$ is near -1, and it grows very large when $y_\mathrm{obs}$ and $y_\mathrm{sim}$ distributions are significantly different.

### Rejection sampler

We're now have the ingredients we need to create a *rejection sampler*, which will follow this algorithm: 

  1. Perturb initial/previous parameters $\theta$ by a small amount to generate new trial parameters $\theta^*$
  
  2. If the trial parameters $\theta^*$ are drawn from within the prior, continue, else return to (1)
  
  3. Generate an example dataset $y_\mathrm{sim}$ using your model $\mathcal{M}$ 
  
  4. Compute _distance_ between the simulated and observed datasets $\rho(y_\mathrm{obs}, y_\mathrm{sim})$
  
  5. For some tolerance $h$, accept the step ($\theta^* = \theta$) if distance $\rho(y_\mathrm{obs}, y_\mathrm{sim}) \leq h$
  
  6. Return to step (1)

In [ ]:
def lnprior(theta): 
    """
    Define a prior probability, which simply requires 
    that -10 < mu_1, mu_2 < 20 and 0 < sigma < 10 and
    0 < fraction < 1.
    """
    mean1, mean2, std, fraction = theta
    
    if -10 < mean1 < 20 and -10 < mean2 < 20 and 0 < std < 10 and 0 <= fraction <= 1: 
        return 0
    return -np.inf

def propose_step(theta, scale): 
    """
    Propose new step: perturb the previous step
    by adding random-normal values to the previous step
    """
    return theta + scale * np.random.randn(len(theta))

def simulate_dataset(theta): 
    """
    Simulate a dataset by generating a bimodal distribution
    with means mu_1, mu_2 and standard deviation sigma
    """
    mean1, mean2, std, fraction = theta
    return np.concatenate([mean1 + std * np.random.randn(int((1-fraction) * len(x))), 
                           mean2 + std * np.random.randn(int(fraction * len(x)))])

def summary_stats(y_obs, y_sim):
    """
    Compute the Anderson-Darling statistic as the distance
    metric between the simulated observations y_sim and the 
    observations y_obs
    """
    distance = anderson_ksamp([y_sim, y_obs]).statistic
    
    return distance

In [ ]:
def rejection_sampler(theta, h, n_steps, scale=0.1, quiet=False, 
                      y_obs=y_obs, prior=lnprior, 
                      simulate_y=simulate_dataset):
    """
    Follow algorithm written above for a simple rejection sampler. 
    """
    # Some bookkeeping variables:
    accepted_steps = 0
    total_steps = 0
    samples = np.zeros((n_steps, len(theta)))
    printed = set()

    while accepted_steps < n_steps: 

        # Make a simple "progress bar":
        if not quiet:
            if accepted_steps % 1000 == 0 and accepted_steps not in printed:
                printed.add(accepted_steps)
                print(f'Sample {accepted_steps} of {n_steps}')

        # Propose a new step:
        new_theta = propose_step(theta, scale)

        # If proposed step is within prior: 
        if np.isfinite(prior(new_theta)): 

            # Generate a simulated dataset from new parameters
            y_sim = simulate_y(new_theta)

            # Compute distance between simulated dataset
            # and the observations
            distance = summary_stats(y_obs, y_sim)
            total_steps += 1

            # If distance is less than tolerance `h`, accept step:
            if distance <= h: 
                theta = new_theta
                samples[accepted_steps, :] = new_theta
                accepted_steps += 1

    print(f'Acceptance rate: {accepted_steps/total_steps}')
    return samples

We can now run our rejection sampler for a given value of the tolerance $h$. 

In [ ]:
# Initial step parameters for the mean and std:
theta = [true_mean1, true_mean2, true_std, true_fraction] 

# Number of posterior samples to compute
n_steps = 5000


# `h` is the distance metric threshold for acceptance;
# try values of h between -0.5 and 5
h = 5

samples = rejection_sampler(theta, h, n_steps)

`samples` now contains `n_steps` approximate posterior samples. Let's make a corner plot which shows the results: 

In [ ]:
labels = ['$\mu_1$', '$\mu_2$', '$\sigma$', '$f$']
truths = [true_mean1, true_mean2, true_std, true_fraction]

corner(samples, truths=truths, 
       levels=[0.6], labels=labels, 
       show_titles=True);

You can experiment with the above example by changing the values of from $h=1$, for a more precise and more computationally expensive approximation to the posterior distribution, or to $h=10$ for a faster but less precise estimate of the posterior distribution. 

In practice, a significant fraction of your effort when applying ABC is spent balancing the computational expense of a small $h$ with the precision you need on your posterior approximation.

We can see how the posterior distribution for the standard deviation $\sigma$ changes as we vary $h$: 

In [ ]:
samples_i = []
h_range = [1, 4, 8]

for h_i in h_range: 
    samples_i.append(rejection_sampler(truths, h_i, n_steps, quiet=True))

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(12, 3))

for s_i, h_i in zip(samples_i, h_range): 
    for j, axis in enumerate(ax): 
        axis.hist(s_i[len(s_i)//2:, j], histtype='step', lw=2, 
                  label=f"h={h_i}", density=True, 
                  bins=30)
        axis.set_xlabel(labels[j]) 
        axis.axvline(truths[j], ls='--', color='#4682b4')
ax[0].set_ylabel('Posterior PDF')
plt.legend()

In the plot above, you can see that the posterior distribution for the standard deviation is largest for the largest $h$, and converges to a narrower distribution centered on the correct value as $h$ decreases.

*** 

## A non-Gaussian example

Now let's do an example where things are less Gaussian. Our data will be distributed with a _beta distribution_, according to 

$$f(x; a,b) = \frac{1}{B(\alpha, \beta)} x^{\alpha - 1} (1 - x)^{\beta - 1},$$

where 

$$B(\alpha, \beta) = \int_0^1 t^{\alpha - 1} (1 - t)^{\beta - 1} dt$$

This new distribution has positive parameters $\theta = \{\alpha, \beta\}$ which we can use ABC to infer:

In [ ]:
from numpy.random import beta

np.random.seed(42)

# The alpha and beta parameters are the tuning parameter
# for beta distributions. 
true_a = 15
true_b = 2

y_obs_beta = beta(true_a, true_b, size=len(x));

plt.hist(y_obs_beta, density=True, histtype='step', color='k', lw=3)
plt.xlabel('$y_\mathrm{obs}$', fontsize=20)
plt.show()

In [ ]:
def lnprior_beta(theta):
    a, b = theta
    if 0 < a < 100 and 0 < b < 100:
        return 0
    return -np.inf
    
def propose_step_beta(theta, scale): 
    """
    Propose new step: perturb the previous step
    by adding random-normal values to the previous step
    """ 
    return theta + scale * np.random.randn(len(theta))

def simulate_dataset_beta(theta): 
    """
    Simulate a dataset by generating a bimodal distribution
    with means mu_1, mu_2 and standard deviation sigma
    """
    a, b = theta
    return beta(a, b, size=len(x))

We'll keep the Anderson-Darling statistic as our summary statistic, which is non-parametric and agnostic about the distributions of the two samples it is comparing. We will swap in our new observations, prior, and simulation function, but nothing else changes in the rejection sampling algorithm:

In [ ]:
# `h` is the distance metric threshold for acceptance;
# try values of h between -0.5 and 5
h = 0

samples = rejection_sampler([true_a, true_b], h, 2 * n_steps, 
                            y_obs=y_obs_beta,
                            prior=lnprior_beta,
                            simulate_y=simulate_dataset_beta)

In [ ]:
labels_beta = [r'$\alpha$', r'$\beta$']
truths_beta = [true_a, true_b]

corner(samples, labels=labels_beta, 
       truths=truths_beta,
       levels=[0.6])
plt.show()

Let's see how random draws from the posterior distributions for $\alpha$ and $\beta$ compare with the observations: 

In [ ]:
props = dict(bins=25, range=[0.5, 1], 
             histtype='step', density=True)

for i in np.random.randint(0, len(samples), size=100): 
    plt.hist(beta(*samples[i, :], size=len(x)), 
             alpha=0.3, color='DodgerBlue', **props)
    
plt.hist(y_obs_beta, color='k', lw=3, **props)
plt.xlabel('$y_\mathrm{obs}, y_\mathrm{sim}$', fontsize=20)

The black histogram is the set of observations $y_\mathrm{obs}$. Shown in blue are various draws from beta distributions with the parameters $\alpha$ and $\beta$ drawn randomly from the posterior distributions from the previous rejection sampling. You can see that the simulated (blue) histograms are "non-rejectable approximations" to the observations (black).